<a href="https://colab.research.google.com/github/hany606/PAI_Fall21IU/blob/main/Assignment3_PAI_Hany_Hamed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment3

Student name: Hany Hamed

Create a Babel Fish

# Source:

* https://colab.research.google.com/github/NVIDIA/NeMo/blob/v1.0.0/tutorials/AudioTranslationSample.ipynb#scrollTo=tdUqxeUEA8nw

* https://datascience.stackexchange.com/questions/92407/audio-not-saving-in-google-colab

# Setup & import

In [7]:
from IPython.display import display, Audio, clear_output, Javascript
from google.colab import output
from base64 import b64decode

from IPython.utils import io
import ipywidgets as widgets
import numpy as np
from time import sleep

In [2]:
# Setup Nemo
BRANCH = 'v1.0.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

clear_output()

In [3]:
import nemo # Nemo library
import nemo.collections.asr as nemo_asr # Speech Recognition
import nemo.collections.nlp as nemo_nlp # NLP
import nemo.collections.tts as nemo_tts   # TTS

[NeMo W 2021-09-07 19:51:44 optimizers:46] Apex was not found. Using the lamb optimizer will error out.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


[NeMo W 2021-09-07 19:51:52 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text_dali.AudioToCharDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [4]:
nemo_asr.models.EncDecCTCModel.list_available_models()
clear_output()

# Speech Recognition model - QuartzNet trained on Russian part of MCV 6.0
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_ru_quartznet15x5").cuda()
clear_output()

# Neural Machine Translation model
nmt_model = nemo_nlp.models.MTEncDecModel.from_pretrained(model_name='nmt_ru_en_transformer6x6').cuda()
clear_output()

# Spectrogram generator which takes text as an input and produces spectrogram
spectrogram_generator = nemo_tts.models.Tacotron2Model.from_pretrained(model_name="tts_en_tacotron2").cuda()
clear_output()

# Vocoder model which takes spectrogram and produces actual audio
vocoder = nemo_tts.models.WaveGlowModel.from_pretrained(model_name="tts_waveglow_88m").cuda()
clear_output()

In [11]:
SAMPLE_RATE = 22050

# 
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

# Russian audio to Text
def ru_audio_txt(audio="audio.wav"):
  russian_text = quartznet.transcribe([audio])
  return russian_text

# Russian text to English
def ru2en(russian_text):
  english_text = nmt_model.translate(russian_text)
  return english_text

# Text to speech
def tts(text):
  parsed = spectrogram_generator.parse(text)
  spectrogram = spectrogram_generator.generate_spectrogram(tokens=parsed)
  audio = vocoder.convert_spectrogram_to_audio(spec=spectrogram)
  return audio.to('cpu').numpy()

# Function to record an audio
def _record_audio(sec):
  clear_output()
  print("Start recording")
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.wav','wb') as f:
    f.write(b)
  print("Finished recording")

# Function to record, to speech, and translate
def record_translate_play(_,audio="audio.wav"):
  _record_audio(record_seconds)
  display(Audio(audio, rate=SAMPLE_RATE, autoplay=True))
  russian_txt = ru_audio_txt()
  print("Russian text:")
  print(russian_txt)
  english_txt = ru2en(russian_txt)
  print("Translation to English text:")
  print(english_txt)
  audio_english = tts(english_txt[0])
  sleep(5)
  display(Audio(audio_english, rate=SAMPLE_RATE, autoplay=True))

# Babal Fish


In [13]:
record_seconds =   7#@param {type:"number", min:1, max:10, step:1}

button = widgets.Button(description="Record and see! (Babel Fish)")
button.on_click(record_translate_play)
display(button)

Start recording


<IPython.core.display.Javascript object>

Finished recording


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo E 2021-09-07 19:57:32 segment:148] Loading audio via SoundFile raised RuntimeError: `Error opening 'audio.wav': File contains data in an unknown format.`. NeMo will fallback to loading via pydub.


Russian text:
['привет ня зовут дони я говорю поруский извинит пожас']
Translation to English text:
['hello my name doni i say prussian sorry fear']
